# Loading real data

In [1]:
import random
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.metrics
import scipy as sp

In [9]:
"""
Created on Tue Nov  6 10:06:52 2018

@author: yandexdataschool

Original Code found in:
https://github.com/yandexdataschool/roc_comparison

updated: Raul Sanchez-Vazquez
"""

import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#german = pd.read_csv("drive/My Drive/Discrétisation ICLR19/data_iclr19/german.csv",sep=";",na_values=['#N/A', '#N/A', 'N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null','.'])

In [4]:
column_names = ['A1',
                'A2',
                'A3',
                'A4',
                'A5',
                'A6',
                'A7',
                'A8',
                'A9',
                'A10',
                'A11',
                'A12',
                'A13',
                'A14',
                'A15',
               'A16',
               'A17',
               'A18',
               'A19',
               'A20',
               'target']

german = pd.read_csv(
    "~/Google Drive/Discrétisation ICLR19/opendata/german.data",
    sep="\s",
    names = column_names,
    header= None,
    index_col = False,
    na_values=[
        '#N/A', '#N/A', 'N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan',
        '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null',
        '.'
    ])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [5]:
german.target = german.target-1

In [6]:
german.dropna(inplace=True)

In [7]:
german.reset_index(inplace=True, drop=True)

# Establishing 1st benchmark: naïve logistic regression

## Logistic Regression

### Label Encoding

In [8]:
german_label_encoders = []

german_encoded = german.copy()

for j in [
        'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'
]:
    temp = sk.preprocessing.LabelEncoder()
    temp.fit(german[j].astype(str))
    german_label_encoders.append(temp)
    german_encoded[j] = temp.transform(german[j].astype(str))

### One-hot encoding

In [11]:
german_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
german_one_hot_encoder.fit(german_encoded[[
        'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'
]])
german_one_hot_encoded = german_encoded.copy()
german_one_hot_encoded.drop(
        ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'],
    axis=1,
    inplace=True)
german_one_hot_encoded = pd.concat(
    [
        german_one_hot_encoded,
        pd.DataFrame(
            german_one_hot_encoder.transform(german_encoded[[
        'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'
            ]]),
            index=german_one_hot_encoded.index)
    ],
    axis=1)

### Data split

In [12]:
import sklearn.model_selection

In [13]:
german_features_train, german_features_test, german_perf_train, german_perf_test = sk.model_selection.train_test_split(
    german_one_hot_encoded.drop('target', axis=1),
    german_one_hot_encoded.target,
    test_size=0.33,
    random_state=1)

In [14]:
german_nn_features_train = german_encoded.iloc[
    german_features_train.index, :].drop(
        'target', axis=1)
german_nn_features_test = german_encoded.iloc[german_features_test.index, :].drop(
    'target', axis=1)
german_nn_perf_train = german_encoded.iloc[
    german_features_train.index, :].target
german_nn_perf_test = german_encoded.iloc[german_features_test.index, :].target

### LR on train data

In [15]:
import sklearn.linear_model

In [17]:
german_naive_LR = sk.linear_model.LogisticRegression(C=1e20, tol=1e-8, solver="newton-cg")
german_naive_LR.fit(
    german_features_train[german_features_train.isna().sum(axis=1) == 0],
    german_perf_train[german_features_train.isna().sum(axis=1) == 0])

LogisticRegression(C=1e+20, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=1e-08, verbose=0, warm_start=False)

### Application of learnt LR on test data

In [18]:
2 * sk.metrics.roc_auc_score(
    german_perf_test,
    german_naive_LR.predict_proba(germaan_features_test)[:, 1]) - 1

0.5210997442455243

In [19]:
alpha = .95
y_pred = german_naive_LR.predict_proba(german_features_test)[:, 1]
y_true = german_perf_test

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)

Gini: 0.5210997442455245
AUC COV: 0.0008542128443291504
95% Gini CI: [0.40653232 0.63566716]


# Establishing 2nd benchmark: MDLP disc + Chi2 grouping

## MDLP disc

In [20]:
from mdlp.discretization import MDLP
transformer_cont_german = MDLP()

In [21]:
transformer_cont_german.fit(
    german_nn_features_train[[
        'A2','A5','A8','A11','A13','A16','A18'
    ]], german_nn_perf_train)

MDLP(continuous_features=None, min_depth=0, random_state=None, shuffle=True)

In [22]:
score_german_MDLP_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
score_german_MDLP_one_hot_encoder.fit(
    transformer_cont_german.transform(german_nn_features_train[[
        'A2','A5','A8','A11','A13','A16','A18'
    ]]))

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=False)

## Grouping

In [23]:
def chi2_test(liste):
    try:
        return sp.stats.chi2_contingency(liste)[1]
    except Exception:
        return 1

In [24]:
german_train_grouped = german.iloc[german_features_train.index, :].copy()
d = dict((x, []) for x in ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'])

for var in ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']:
    
    german_train_grouped[var] = german_train_grouped[var].astype(str)
    d[var] = [x for x in np.unique(german_train_grouped[var])]
    p_value = 1

    while(p_value>0.05):
        if len(np.unique(german_train_grouped[var]))>1:
            freq_table = german_train_grouped.groupby([var,'target']).size().reset_index()
            liste_paires_modalites = [[a,b] for a in np.unique(german_train_grouped[var]) for b in np.delete(np.unique(german_train_grouped[var]),np.where(np.unique(german_train_grouped[var])==a))]
            liste_chi2 = [chi2_test([freq_table.iloc[np.in1d(freq_table[var],pair[0]),2],freq_table.iloc[np.in1d(freq_table[var],pair[1]),2]]) for pair in liste_paires_modalites]
            p_value = max(liste_chi2)
        else: break

        if (p_value>0.05 and len(np.unique(german_train_grouped[var]))>1):
            german_train_grouped[var].iloc[np.in1d(german_train_grouped[var],liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))])] = liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][0] + ' - ' + liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][1]
            d[var].remove(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][0])
            d[var].remove(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][1])
            d[var].append(str(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][0] + ' - ' + liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))][1]))
            print('Feature '+var+ ' - levels merged : '+str(liste_paires_modalites[np.argmax(np.equal(liste_chi2,p_value))]))
        else: break


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Feature A1 - levels merged : ['A13', 'A14']
Feature A3 - levels merged : ['A32', 'A33']
Feature A3 - levels merged : ['A30', 'A31']
Feature A4 - levels merged : ['A410', 'A44']
Feature A4 - levels merged : ['A410 - A44', 'A45']
Feature A4 - levels merged : ['A410 - A44 - A45', 'A46']
Feature A4 - levels merged : ['A40', 'A410 - A44 - A45 - A46']
Feature A4 - levels merged : ['A43', 'A48']
Feature A4 - levels merged : ['A40 - A410 - A44 - A45 - A46', 'A49']
Feature A4 - levels merged : ['A40 - A410 - A44 - A45 - A46 - A49', 'A42']
Feature A6 - levels merged : ['A61', 'A62']
Feature A6 - levels merged : ['A63', 'A65']
Feature A6 - levels merged : ['A64', 'A63 - A65']
Feature A7 - levels merged : ['A71', 'A73']
Feature A7 - levels merged : ['A74', 'A75']
Feature A7 - levels merged : ['A71 - A73', 'A74 - A75']
Feature A9 - levels merged : ['A93', 'A94']
Feature A9 - levels merged : ['A91', 'A92']
Feature A10 - levels merged : ['A101', 'A103']
Feature A10 - levels merged : ['A102', 'A101 - 

## Test time

In [25]:
german_train_mdlp = score_german_MDLP_one_hot_encoder.transform(
    transformer_cont_german.transform(german_nn_features_train[[
        'A2','A5','A8','A11','A13','A16','A18'
    ]]))

In [26]:
german_train_grouped_label_encoders = []

german_train_grouped_encoded = german_train_grouped.copy()

for j in ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']:
    temp = sk.preprocessing.LabelEncoder()
    temp.fit(german_train_grouped_encoded[j].astype(str))
    german_train_grouped_label_encoders.append(temp)
    german_train_grouped_encoded[j] = temp.transform(german_train_grouped_encoded[j].astype(str))

In [27]:
score_german_CHI2_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")

score_german_CHI2_one_hot_encoder.fit(
        german_train_grouped_encoded[[
            'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'
        ]])

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=False)

In [29]:
german_train_chi2 = score_german_CHI2_one_hot_encoder.transform(german_train_grouped_encoded[[
                    'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'

        ]])

In [30]:
german_adhoc_train = np.concatenate((german_train_chi2,german_train_mdlp),axis=1)

In [31]:
german_adhoc_LR = sk.linear_model.LogisticRegression(C=1e20, tol=1e-8, solver="newton-cg")
german_adhoc_LR.fit(
    german_adhoc_train,
    german_train_grouped_encoded['target'])

LogisticRegression(C=1e+20, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=1e-08, verbose=0, warm_start=False)

In [32]:
german_test_grouped = german.iloc[german_features_test.index, :].copy()

for var in ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']:
    
    german_test_grouped[var] = german_test_grouped[var].astype(str)

    for x in d[var]:
        if x.find(' - ')>-1:
            liste_modalites = x.split(' - ')
            german_test_grouped[var].iloc[np.in1d(german_test_grouped[var],liste_modalites)] = x
        

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
german_test_grouped_encoded = german_test_grouped.copy()

for j in ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']:
    indice = ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'].index(j)
    german_test_grouped_encoded[j] = german_train_grouped_label_encoders[indice].transform(german_test_grouped_encoded[j].astype(str))
    

In [34]:
german_test_chi2 = score_german_CHI2_one_hot_encoder.transform(german_test_grouped_encoded[[
                    'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'

        ]])

In [35]:
german_test_mdlp = score_german_MDLP_one_hot_encoder.transform(
    transformer_cont_german.transform(german_nn_features_test[[
        'A2','A5','A8','A11','A13','A16','A18'
    ]]))

In [36]:
german_adhoc_test = np.concatenate(
    (german_test_chi2, german_test_mdlp), axis=1)

In [38]:
2 * sk.metrics.roc_auc_score(
    german_test_grouped_encoded
              ['target'],
    german_adhoc_LR.predict_proba(german_adhoc_test)[:, 1]) - 1

0.5457161125319692

In [39]:
alpha = .95
y_pred = german_adhoc_LR.predict_proba(german_adhoc_test)[:, 1]
y_true = german_test_grouped_encoded['target']

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)

Gini: 0.5457161125319694
AUC COV: 0.0008128572147174783
95% Gini CI: [0.43395641 0.65747581]
